In [1]:
#Imports
!pip install emoji
import emoji
from sklearn.model_selection import train_test_split 
from sklearn.utils import shuffle
from sklearn.metrics import classification_report, plot_confusion_matrix, accuracy_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB, BernoulliNB, GaussianNB
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import json
import numpy as np
import warnings
warnings.filterwarnings('ignore')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 175 kB 26.5 MB/s 
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=3bf0cec846d4dd1e7bf85d56d578200444a9fd02877c43f00479f4a20f56f4cf
  Stored in directory: /root/.cache/pip/wheels/8a/4e/b6/57b01db010d17ef6ea9b40300af725ef3e210cb1acfb7ac8b6
Successfully built emoji


In [2]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


##Stop words and vectorizing
https://github.com/6/stopwords-json

TfidfVectorizer or countvectorizer? now using CV, could be other type too

#NAIVE BAYES


In [3]:
def naive_bayes(i, balance, lang, emo_hash, test, nbtype, print_metrics, vectorizer):
  if i == 0:#
    i = 1#
  if balance:#
    #we save our txt files in a general path, I later save them into their specific folders
    path = "/content/drive/MyDrive/TFG/data/accuracy_data/NB_" + lang + "_" + emo_hash + "_" + str(100-(test*100)) + str(test*100) +"TrainTest_" + nbtype + "_" + vectorizer + "_BALANCED.txt"#
  else:#
    path = "/content/drive/MyDrive/TFG/data/accuracy_data/NB_" + lang + "_" + emo_hash + "_" + str(100-(test*100)) + str(test*100) +"TrainTest_" + nbtype + "_" + vectorizer + "_NOT_BALANCED.txt"#
  acc_file = open(path,"w+") #
  for x in range(i):#
    #Data loading---------------------------------GENERAL--CODE-----------------------------------------------
    df = pd.read_csv('/content/drive/MyDrive/TFG/data/final_data/' + emo_hash + '_' + lang + '_data.csv', encoding='utf8', engine='python')
    #Final row cleansing
    df = df[(df['hate speech'] == 0) | (df['hate speech'] == 1)]
    df = df.dropna()
    df = shuffle(df)
    #Balancing data
    if balance: 
      pos_rows = len(df[df["hate speech"] == True].index)
      fraction_to_delete = 1 - (pos_rows/ (df.shape[0]-pos_rows))
      df = df.drop(df[df['hate speech'] == 0].sample(frac=fraction_to_delete).index)
    #Train and test split
    X, y = df.text.fillna(' '), df["hate speech"]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = test)
    #Vectorizing
    with open("/content/drive/MyDrive/TFG/data/stopwords/" + lang + "_stopwords.json", "r") as f:
      json_text = f.read()
    stopwords = list(json.loads(json_text))
    if vectorizer == "CountVectorizer":
      vect = CountVectorizer(stop_words = stopwords, binary = True)
    else:
      vect = TfidfVectorizer(stop_words = stopwords, binary = True) # tfidf here
    X_train_vect = vect.fit_transform(X_train)
    X_test_vect = vect.transform(X_test)
    #Model building---------------------------------GENERAL--CODE-----------------------------------------------
    if nbtype == "Bernoulli":
      model = BernoulliNB()
    else:
      model = MultinomialNB()
    model.fit(X_train_vect, y_train)
    #Plot printing
    if print_metrics:
      y_pred = model.predict(X_test_vect)
      acc_file.write(str(accuracy_score(y_test, y_pred) * 100) + "\n")#
      # print(classification_report(y_test,y_pred))
      # print("-------------------------------------------------------")
      print("Accuracy score for Naive Bayes is: ", accuracy_score(y_test, y_pred) * 100, '%')
      # print("-------------------------------------------------------")
      # #cmap options: YlOrRd, GnBu, RdPu, YlGn
      # plot_confusion_matrix(model, X_test_vect, y_test, normalize='true', cmap ="GnBu")
  acc_file.close()#
  return model, vect

#SUPPORT VECTOR MACHINES
Using SVM classifiers for text classification tasks might be a really good idea, especially if the training data available is not much (~ a couple of thousand tagged samples).

In [4]:
def support_vector_machine(i, balance, lang, emo_hash, test, kernel, c, print_metrics, vectorizer):
  if i == 0:#
    i = 1#
  if balance:#
    #we save our txt files in a general path, I later save them into their specific folders
    path = "/content/drive/MyDrive/TFG/data/accuracy_data/SVM_" + lang + "_" + emo_hash + "_" + str(100-(test*100)) + str(test*100) +"TrainTest_" + kernel + "Kernel_" + str(c) + "C_" + vectorizer + "_BALANCED.txt"#
  else:#
    path = "/content/drive/MyDrive/TFG/data/accuracy_data/SVM_" + lang + "_" + emo_hash + "_" + str(100-(test*100)) + str(test*100) +"TrainTest_" + kernel + "Kernel_" + str(c) + "C_" + vectorizer + "_NOT_BALANCED.txt"#
  acc_file = open(path,"w+") #
  for x in range(i):#
    #Data loading---------------------------------GENERAL--CODE-----------------------------------------------
    df = pd.read_csv('/content/drive/MyDrive/TFG/data/final_data/' + emo_hash + '_' + lang + '_data.csv', encoding='utf8', engine='python')
    #Final row cleansing
    df = df[(df['hate speech'] == 0) | (df['hate speech'] == 1)]
    df = df.dropna()
    df = shuffle(df)
    #Balancing data
    if balance: 
      pos_rows = len(df[df["hate speech"] == True].index)
      fraction_to_delete = 1 - (pos_rows/ (df.shape[0]-pos_rows))
      df = df.drop(df[df['hate speech'] == 0].sample(frac=fraction_to_delete).index)
    #Train and test split
    X, y = df.text.fillna(' '), df["hate speech"]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = test)
    #Vectorizing
    with open("/content/drive/MyDrive/TFG/data/stopwords/" + lang + "_stopwords.json", "r") as f:
      json_text = f.read()
    stopwords = list(json.loads(json_text))
    if vectorizer == "CountVectorizer":
      vect = CountVectorizer(stop_words = stopwords, binary = True)
    else:
      vect = TfidfVectorizer(stop_words = stopwords, binary = True) # tfidf here
    X_train_vect = vect.fit_transform(X_train)
    X_test_vect = vect.transform(X_test)
    #Model building---------------------------------GENERAL--CODE-----------------------------------------------
    model = SVC(kernel=kernel, C=c)
    model.fit(X_train_vect, y_train)
    #Plot printing
    if print_metrics:
      y_pred = model.predict(X_test_vect)
      acc_file.write(str(accuracy_score(y_test, y_pred) * 100) + "\n")#
      # print(classification_report(y_test,y_pred))
      # print("-------------------------------------------------------")
      print("Accuracy score for SVC is: ", accuracy_score(y_test, y_pred) * 100, '%')
      # print("-------------------------------------------------------")
      # #cmap options: YlOrRd, GnBu, RdPu, YlGn
      # plot_confusion_matrix(model, X_test_vect, y_test, normalize='true', cmap ="YlOrRd")
  acc_file.close()#
  return model, vect

#LOGISTIC REGRESSION


In [5]:
def logistic_regression(i, balance, lang, emo_hash, test, solver, c, print_metrics, vectorizer):
  if i == 0:#
    i = 1#
  if balance:#
    #we save our txt files in a general path, I later save them into their specific folders
    path = "/content/drive/MyDrive/TFG/data/accuracy_data/LR_" + lang + "_" + emo_hash + "_" + str(100-(test*100)) + str(test*100) +"TrainTest_" + solver + "Solver_" + str(c) + "C_" + vectorizer + "_BALANCED.txt"#
  else:#
    path = "/content/drive/MyDrive/TFG/data/accuracy_data/LR_" + lang + "_" + emo_hash + "_" + str(100-(test*100)) + str(test*100) +"TrainTest_" + solver + "Solver_" + str(c) + "C_" + vectorizer + "_NOT_BALANCED.txt"#
  acc_file = open(path,"w+") #
  for x in range(i):#
    #Data loading---------------------------------GENERAL--CODE-----------------------------------------------
    df = pd.read_csv('/content/drive/MyDrive/TFG/data/final_data/' + emo_hash + '_' + lang + '_data.csv', encoding='utf8', engine='python')
    #Final row cleansing
    df = df[(df['hate speech'] == 0) | (df['hate speech'] == 1)]
    df = df.dropna()
    df = shuffle(df)
    #Balancing data
    if balance: 
      pos_rows = len(df[df["hate speech"] == True].index)
      fraction_to_delete = 1 - (pos_rows/ (df.shape[0]-pos_rows))
      df = df.drop(df[df['hate speech'] == 0].sample(frac=fraction_to_delete).index)
    #Train and test split
    X, y = df.text.fillna(' '), df["hate speech"]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = test)
    #Vectorizing
    with open("/content/drive/MyDrive/TFG/data/stopwords/" + lang + "_stopwords.json", "r") as f:
      json_text = f.read()
    stopwords = list(json.loads(json_text))
    if vectorizer == "CountVectorizer":
      vect = CountVectorizer(stop_words = stopwords, binary = True)
    else:
      vect = TfidfVectorizer(stop_words = stopwords, binary = True) # tfidf here
    X_train_vect = vect.fit_transform(X_train)
    X_test_vect = vect.transform(X_test)
    #Model building---------------------------------GENERAL--CODE-----------------------------------------------
    model = LogisticRegression(solver=solver, C=c)
    model.fit(X_train_vect, y_train)
    #Plot printing
    if print_metrics:
      y_pred = model.predict(X_test_vect)
      acc_file.write(str(accuracy_score(y_test, y_pred) * 100) + "\n")#
      # print(classification_report(y_test,y_pred))
      # print("-------------------------------------------------------")
      print("Accuracy score for Logistic Regression is: ", accuracy_score(y_test, y_pred) * 100, '%')
      # print("-------------------------------------------------------")
      # #cmap options: YlOrRd, GnBu, RdPu, YlGn
      # plot_confusion_matrix(model, X_test_vect, y_test, normalize='true', cmap ="RdPu")
  acc_file.close()#
  return model, vect

------------------------------------------------------------------------
#Accuracy checker and hate speech detectors

In [41]:
#@title 1. Choose a model and language

Language = 'Spanish'  #@param ["Spanish", "Italian", "Portuguese", "English"]

map_lang_data = {
    'Spanish':
        'spanish',
    'Italian':
        'italian',
    'Portuguese':
        'portuguese',
    'English':
        'english',
}

Emojis_Hashtags = 'No emojis or hashtags'  #@param ["All emojis and hashtags", "Filtered emojis and hashtags", "No emojis or hashtags"]

map_emojhash_data = {
    'All emojis and hashtags':
        'mantained',
    'Filtered emojis and hashtags':
        'filtered',
    'No emojis or hashtags':
        'removed',
}

Train_Test_Split = '70/30'  #@param ['60/40', '70/30', '80/20']

map_test_split = {
    '60/40':
        0.4,
    '70/30':
        0.3,
    '80/20':
        0.2,
}

Model = 'SVM linear kernel large C'  #@param ["Bernoulli NB", "Multinomial NB", "SVM linear kernel small C", "SVM linear kernel standard C", "SVM linear kernel large C", "SVM RBF kernel small C", "SVM RBF kernel standard C", "SVM RBF kernel large C", "LR liblinear solver small C", "LR liblinear solver standard C","LR liblinear solver large C", "LR lbfgs solver small C", "LR lbfgs solver standard C", "LR lbfgs solver large C"]

TF_IDF = False #@param {type:"boolean"}
Print_model_metrics = True #@param {type:"boolean"}
Test_Iterations = 30 #@param {type:"slider", min:1, max:50, step:1}
Balance_data = True #@param {type:"boolean"}

map_vectorizer = {
    True:
        "TfidfVectorizer",
    False:
        "CountVectorizer",
}

lang = map_lang_data[Language]
emo_hash = map_emojhash_data[Emojis_Hashtags]
test = map_test_split[Train_Test_Split]
vectorizer = map_vectorizer[TF_IDF]
if TF_IDF:
  vectorizer_word = "with"
else:
  vectorizer_word = "without"

print((Language + " data with " + Emojis_Hashtags +" using model " + Model + ", " + vectorizer_word + " TF-IDF and train/test split of " + Train_Test_Split).upper())
print("-------------------------------------------------------------------------------------------------------------------------------")

if Model == "SVM linear kernel small C":
  chosen_model, vect = support_vector_machine(Test_Iterations, Balance_data, lang, emo_hash, test, "linear", 0.1, Print_model_metrics, vectorizer)
elif Model == "SVM linear kernel standard C":
  chosen_model, vect = support_vector_machine(Test_Iterations, Balance_data, lang, emo_hash, test, "linear", 1, Print_model_metrics, vectorizer)
elif Model == "SVM linear kernel large C":
  chosen_model, vect = support_vector_machine(Test_Iterations, Balance_data, lang, emo_hash, test, "linear", 10, Print_model_metrics, vectorizer)
elif Model == "SVM RBF kernel small C":
  chosen_model, vect = support_vector_machine(Test_Iterations, Balance_data, lang, emo_hash, test, "rbf", 0.1, Print_model_metrics, vectorizer)
elif Model == "SVM RBF kernel standard C":
  chosen_model, vect = support_vector_machine(Test_Iterations, Balance_data, lang, emo_hash, test, "rbf", 1, Print_model_metrics, vectorizer)
elif Model == "SVM RBF kernel large C":
  chosen_model, vect = support_vector_machine(Test_Iterations, Balance_data, lang, emo_hash, test, "rbf", 10, Print_model_metrics, vectorizer)

elif Model == "Bernoulli NB":
  chosen_model, vect = naive_bayes(Test_Iterations, Balance_data, lang, emo_hash, test, "Bernoulli", Print_model_metrics, vectorizer)
elif Model == "Multinomial NB":
  chosen_model, vect = naive_bayes(Test_Iterations, Balance_data, lang, emo_hash, test, "Multinomial", Print_model_metrics, vectorizer)

elif Model == "LR liblinear solver small C":
  chosen_model, vect = logistic_regression(Test_Iterations, Balance_data, lang, emo_hash, test, "liblinear", 0.1, Print_model_metrics, vectorizer)
elif Model == "LR liblinear solver standard C":
  chosen_model, vect = logistic_regression(Test_Iterations, Balance_data, lang, emo_hash, test, "liblinear", 1, Print_model_metrics, vectorizer)
elif Model == "LR liblinear solver large C":
  chosen_model, vect = logistic_regression(Test_Iterations, Balance_data, lang, emo_hash, test, "liblinear", 10, Print_model_metrics, vectorizer)
elif Model == "LR lbfgs solver small C":
  chosen_model, vect = logistic_regression(Test_Iterations, Balance_data, lang, emo_hash, test, "lbfgs", 0.1, Print_model_metrics, vectorizer)
elif Model == "LR lbfgs solver standard C":
  chosen_model, vect = logistic_regression(Test_Iterations, Balance_data, lang, emo_hash, test, "lbfgs", 1, Print_model_metrics, vectorizer)
elif Model == "LR lbfgs solver large C":
  chosen_model, vect = logistic_regression(Test_Iterations, Balance_data, lang, emo_hash, test, "lbfgs", 10, Print_model_metrics, vectorizer)

SPANISH DATA WITH NO EMOJIS OR HASHTAGS USING MODEL SVM LINEAR KERNEL LARGE C, WITHOUT TF-IDF AND TRAIN/TEST SPLIT OF 70/30
-------------------------------------------------------------------------------------------------------------------------------
Accuracy score for SVC is:  70.38473009245452 %
Accuracy score for SVC is:  69.28124067998806 %
Accuracy score for SVC is:  69.01282433641515 %
Accuracy score for SVC is:  70.23560990158067 %
Accuracy score for SVC is:  70.23560990158067 %
Accuracy score for SVC is:  70.44437816880405 %
Accuracy score for SVC is:  70.26543393975544 %
Accuracy score for SVC is:  69.43036087086192 %
Accuracy score for SVC is:  70.29525797793022 %
Accuracy score for SVC is:  68.4759916492693 %
Accuracy score for SVC is:  70.62332239785268 %
Accuracy score for SVC is:  70.1163137488816 %
Accuracy score for SVC is:  69.96719355800775 %
Accuracy score for SVC is:  70.20578586340591 %
Accuracy score for SVC is:  70.20578586340591 %
Accuracy score for SVC is:  69

In [ ]:
#@title 2. Write some text to see if it contains hate speech...
text = "" #@param {type:"string"}
text = text.encode('utf-16','surrogatepass').decode('utf-16')
#demojize possible emojis
emo_full_list = ''.join(c for c in text if c in emoji.UNICODE_EMOJI['en'])
emo_unique_list = np.unique(list(emo_full_list))
for k in emo_unique_list:
    text = text.replace(k, " " + emoji.demojize(k) + " ")

prediction = chosen_model.predict(vect.transform([text]))

if prediction == 1 or prediction == "hate speech":
  print("YES, This text contains hate speech")
else:
  print("NO, This text does not contain hate speech")

YES, This text contains hate speech
